## Download data from EDD

data was previously saved as a CSV using the following code

```python
!pip install --user edd-utils
from edd_utils import login, export_study
study_slug = 'pputida_wt_cj522_gb032_gb045_gb062'
edd_server = 'edd.agilebiofoundry.org'

session = login(edd_server=edd_server, user='pstjohn')
df = export_study(session, study_slug, edd_server=edd_server)
```

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import gzip
import pickle

from tqdm import tqdm
import numpy as np
import pandas as pd
df = pd.read_csv('pputida_wt_cj522_gb032_gb045_gb062.csv.gz')

In [2]:
df.Protocol.unique()

array(['PNNL Global Proteomics', 'Targeted Proteomics',
       'PNNL Global Metabolomics (extracellular)',
       'PNNL Global Metabolomics (intracellular)'], dtype=object)

## Convert EDD identifiers to BIGG identifiers (proteins and metabolites)

In [3]:
# Jeremy Z. provided these uniprot matches
protein_mapping = pd.read_csv('P_putida_KT2440_uniprot_protein_refs.tsv', sep='\t')
protein_mapping.head()

,FrameId,Locus,GeneSymbol,ProductName,GenbankAccession.version,Entrez,UniprotAccession,UniprotName
0,G18UU-22920-MONOMER,PP_5387,cusA,"probable copper efflux transporter, CzcA family",AAN70952.1,24987202,Q88BZ6,Q88BZ6_PSEPK
1,G18UU-22919-MONOMER,PP_5386,cusB,Probable copper RND efflux membrane fusion pro...,AAN70951.1,24987201,Q88BZ7,Q88BZ7_PSEPK
2,G18UU-22905-MONOMER,PP_5374,PP_5374,Choline/carnitine/betaine transporter family p...,AAN70939.1,24987188,Q88C09,Q88C09_PSEPK
3,G18UU-22861-MONOMER,PP_5329,PP_5329,"putative phosphate ABC transporter, periplasmi...",AAN70894.1,24987138,Q88C54,Q88C54_PSEPK
4,G18UU-22860-MONOMER,PP_5328,PP_5328,putative phosphate transport system permease p...,AAN70893.2,1001556072,Q88C55,Q88C55_PSEPK


In [4]:
df_protein = df[df.Protocol.str.contains('Proteomics')]
df_protein = df_protein.merge(protein_mapping[['Locus', 'GeneSymbol', 'UniprotAccession']],
                              how='left', left_on='Formal Type', right_on='UniprotAccession')

# For AsbF and AroG enzymes, fill with name
df_protein['Locus'] = df_protein.Locus.fillna(df_protein['Measurement Type'])
df_protein.head()

,Study ID,Study Name,Line ID,Line Name,Line Description,Protocol,Assay ID,Assay Name,Formal Type,Measurement Type,Compartment,Units,Value,Hours,Locus,GeneSymbol,UniprotAccession
0,7882,P.putida_WT_CJ522_GB032_GB045_GB062,8163,WT-glu-R3,NaN,PNNL Global Proteomics,8492,WT-glu-R3,Q88EQ1,NaN,0,intensity,30.08957,24.0,PP_4402,bkdAB,Q88EQ1
1,7882,P.putida_WT_CJ522_GB032_GB045_GB062,8163,WT-glu-R3,NaN,PNNL Global Proteomics,8492,WT-glu-R3,Q88EQ2,2-oxoisovalerate dehydrogenase subunit alpha,0,intensity,28.55962,24.0,PP_4401,bkdAA,Q88EQ2
2,7882,P.putida_WT_CJ522_GB032_GB045_GB062,8163,WT-glu-R3,NaN,PNNL Global Proteomics,8492,WT-glu-R3,Q88EQ6,Flagellar brake protein YcgR,0,intensity,33.36829,24.0,PP_4397,ycgR,Q88EQ6
3,7882,P.putida_WT_CJ522_GB032_GB045_GB062,8163,WT-glu-R3,NaN,PNNL Global Proteomics,8492,WT-glu-R3,Q88EQ7,NaN,0,intensity,31.72792,24.0,PP_4396,PP_4396,Q88EQ7
4,7882,P.putida_WT_CJ522_GB032_GB045_GB062,8163,WT-glu-R3,NaN,PNNL Global Proteomics,8492,WT-glu-R3,Q88EQ8,NaN,0,intensity,29.86861,24.0,PP_4395,flgM,Q88EQ8


In [5]:
# Some of these may be missing the correct annotation
df_protein['Locus'][~df_protein['Locus'].str.startswith('PP_')].unique()

array(['AroG-D146N', '3-hydroxyisobutyrate dehydrogenase', 'Q88GS0_PSEPK',
       'Q88QD9_PSEPK', 'AsbF', 'B3ZVR2_BACCE', 'Q88EL2_PSEPK',
       'Q88GR0_PSEPK', 'Q88HX9_PSEPK', 'Q88KP7_PSEPK'], dtype=object)

In [6]:
# These mappings were made with the help of the chemical translation service,
# https://cts.fiehnlab.ucdavis.edu/, as well as a few manual matches

metabolite_mapping = pd.read_csv('cid_to_bigg_matches.csv')
metabolite_mapping['cid'] = 'cid:' + metabolite_mapping.cid.astype('str')
metabolite_mapping.head()

,cid,db,identifier,metabolite
0,cid:3035456,KEGG,C06473,2dhglcn_c
1,cid:72,KEGG,C00230,34dhbz_c
2,cid:22639876,KEGG,C01353,hco3_c
3,cid:5280518,KEGG,C02480,ccmuac_c
4,cid:10690,KEGG,C00257,glcn_c


In [7]:
df_metabolite = df[df.Protocol.str.contains('Metabolomics')]
df_metabolite = df_metabolite.merge(metabolite_mapping, how='inner', left_on='Formal Type', right_on='cid')
df_metabolite.head()

,Study ID,Study Name,Line ID,Line Name,Line Description,Protocol,Assay ID,Assay Name,Formal Type,Measurement Type,Compartment,Units,Value,Hours,cid,db,identifier,metabolite
0,7882,P.putida_WT_CJ522_GB032_GB045_GB062,8118,WT-gfg-R3,NaN,PNNL Global Metabolomics (extracellular),9393,WT-gfg-R3,cid:3035456,"(3S,4R,5R)-3,4,5,6-tetrahydroxy-2-keto-hexanoi...",2,NaN,3.753492e+06,24.0,cid:3035456,KEGG,C06473,2dhglcn_c
1,7882,P.putida_WT_CJ522_GB032_GB045_GB062,8131,WT-gg-R1,NaN,PNNL Global Metabolomics (extracellular),9394,WT-gg-R1,cid:3035456,"(3S,4R,5R)-3,4,5,6-tetrahydroxy-2-keto-hexanoi...",2,NaN,5.677345e+06,24.0,cid:3035456,KEGG,C06473,2dhglcn_c
2,7882,P.putida_WT_CJ522_GB032_GB045_GB062,8132,WT-gg-R2,NaN,PNNL Global Metabolomics (extracellular),9395,WT-gg-R2,cid:3035456,"(3S,4R,5R)-3,4,5,6-tetrahydroxy-2-keto-hexanoi...",2,NaN,3.880688e+06,24.0,cid:3035456,KEGG,C06473,2dhglcn_c
3,7882,P.putida_WT_CJ522_GB032_GB045_GB062,8133,WT-gg-R3,NaN,PNNL Global Metabolomics (extracellular),9396,WT-gg-R3,cid:3035456,"(3S,4R,5R)-3,4,5,6-tetrahydroxy-2-keto-hexanoi...",2,NaN,4.136200e+06,24.0,cid:3035456,KEGG,C06473,2dhglcn_c
4,7882,P.putida_WT_CJ522_GB032_GB045_GB062,8146,WT-glc-R1,NaN,PNNL Global Metabolomics (extracellular),9397,WT-glc-R1,cid:3035456,"(3S,4R,5R)-3,4,5,6-tetrahydroxy-2-keto-hexanoi...",2,NaN,2.187088e+06,24.0,cid:3035456,KEGG,C06473,2dhglcn_c


In [8]:
# Correct a few metabolites that are only present in the periplasm
df_metabolite.loc[df_metabolite.Protocol == 'PNNL Global Metabolomics (intracellular)', 'metabolite'] = \
    df_metabolite.loc[df_metabolite.Protocol == 'PNNL Global Metabolomics (intracellular)', 'metabolite'].replace({
    '2dhglcn_c': '2dhglcn_p',
    'hdca_c': 'hdca_p',
    'ocdca_c': 'ocdca_p'})

# Correct the compartment for extracellular omics
df_metabolite.loc[df_metabolite.Protocol == 'PNNL Global Metabolomics (extracellular)', 'metabolite'] = \
    df_metabolite.loc[df_metabolite.Protocol == 'PNNL Global Metabolomics (extracellular)', 'metabolite'].str.replace('_[cp]$', '_e')

# Split internal and external metabolomics
df_internal = df_metabolite[df_metabolite.Protocol == 'PNNL Global Metabolomics (intracellular)']
df_external = df_metabolite[df_metabolite.Protocol == 'PNNL Global Metabolomics (extracellular)']

## Load (reduced) metabolic model
* details on model reduction in `model_modifications.ipynb` and `model_compression.ipynb`

In [9]:
import cobra
model = cobra.io.load_json_model('cobrapy_models/putida_gb_newgenes.json')
    
aroG = model.genes.get_by_id('aroG-D146N')
aroG.id = 'AroG_D146N'
aroG.name = 'AroG-D146N'
model.genes.add(cobra.Gene(id='AsbF', name='AsbF'))
model.genes.add(cobra.Gene(id='B3ZVR2_BACCE', name='B3ZVR2_BACCE'))
model.reactions.DHSKDH.gene_reaction_rule = 'PP_2554 or AsbF or B3ZVR2_BACCE'    

medium = model.medium

medium['EX_glcn_e'] = 1.801952
medium['EX_fru_e'] = 0.444581
medium['EX_glc_e'] = 0.146702

model.medium = medium

kos = ['GLCDpp', 'PGI', 'PPC', 'PYK']
for rxn in kos:
    model.reactions.get_by_id(rxn).remove_from_model()
    
model.add_boundary(model.metabolites.s7p_c, type='demand')

model.reactions.SUCCt2_2.remove_from_model()
model.reactions.PPCK.remove_from_model()
model.metabolites.succ_e.remove_from_model(destructive=True)
model.metabolites.pyr_e.remove_from_model(destructive=True)

model.add_boundary(model.metabolites.get_by_id('2dhglcn_c'), type='demand')
model.reactions.DM_2dhglcn_c.lower_bound = 1.666

model.reactions.ATPM.lower_bound = 1.
model.reactions.muconate_sink.id = 'DM_ccmuac_c'

with model:
    model.reactions.GAD2ktpp.lower_bound = 0.1
    model.reactions.GLUN.lower_bound = 0.1
    model.reactions.GND.lower_bound = 0.1
    model.reactions.ICL.lower_bound = 0.1    
    model.reactions.ME2.lower_bound = 0.05
    model.reactions.THD2.lower_bound = .1
    model.reactions.GLUSy.lower_bound = .1
#    model.reactions.PPCK.lower_bound = .1
    model.reactions.PGLCNDH.lower_bound = .1
#    model.reactions.PC.lower_bound = 3
#    model.reactions.MDH.lower_bound = 0.
    
#     model.reactions.EX_pyr_e.lower_bound = 0.1
#     model.reactions.EX_succ_e.lower_bound = 0.1
    model.reactions.DM_s7p_c.lower_bound = 0.25
    
    sol = cobra.flux_analysis.pfba(model)
    
print(sol.fluxes[np.isclose(sol.fluxes, 0)])
    
v_star = sol.fluxes.values    
N = cobra.util.create_stoichiometric_matrix(model)

# Correct negative flux values at the reference state
N[:, v_star < 0] = -1 * N[:, v_star < 0]
v_star = np.abs(v_star)

m_labels = [m.id for m in model.metabolites]
r_labels = [r.id for r in model.reactions]

Series([], Name: fluxes, dtype: float64)


## Load boundary fluxes calculated from spent media
I don't believe this data made it into EDD, this was sent to me from Gayle and processed in a seperate file. Essentially I used the time and OD at collection to fit an exponential growth curve, and then calculate averaged specific uptake and secretion rates in mmol/gDCW*hr

In [10]:
boundary_data = pd.read_csv('boundary_flux_from_external_measurements.csv')
boundary_data = boundary_data.join(boundary_data['Sample Name'].str.extract('^(?P<strain>\S+)\.(?P<media>\S+)\.(?P<replicate>\d+)'))
# boundary_data.head()

## Normalize data and reference to model

In [11]:
boundary_data = boundary_data[boundary_data.strain  != 'KT2440']
boundary_fluxes = boundary_data.groupby(['strain', 'media'])[
    ['growth_rate', 'glucose_uptake', 'fructose_uptake', 'gluconate_uptake',
     '2-ketogluconate_production', 'muconate_production']].mean()

normalized_fluxes = boundary_fluxes.divide(boundary_fluxes.loc[('GB032', 'gfg')])
normalized_fluxes.columns = ['Biomass_Ecoli_core_w_GAM', 'EX_glc_e', 'EX_fru_e', 'EX_glcn_e', 'DM_2dhglcn_c', 'DM_ccmuac_c']

In [12]:
normalized_fluxes.columns.isin(r_labels)

array([ True,  True,  True,  True,  True,  True])

In [13]:
rxn_indexer = pd.Series(r_labels).reset_index().set_index(0)
vn = normalized_fluxes
v_inds = np.array([rxn_indexer.loc[r] for r in vn.columns]).flatten()

#### convert extracellular concentrations into relative fluxes for other species

In [14]:
# df_external_mean = df_external.join(df_external['Assay Name'].str.extract(
#     '^(?P<strain>\S+)-(?P<media>\S+)-(?P<replicate>R\d+)')).pivot_table(
#     values='Value', index=['strain', 'media'], columns='metabolite').drop('WT')

# normalized_external = df_external_mean.divide(df_external_mean.loc[('GB032', 'gfg')])
# model_mets = {m.id for m in model.metabolites}
# normalized_external = normalized_external.loc[:, normalized_external.columns.isin(model_mets)]
# normalized_external = normalized_external.drop(['2dhglcn_e', 'ccmuac_e', 'glcn_e', 'glc__D_e', 'fru_e'], 1).dropna(axis=1)
# normalized_external.columns = 'EX_' + normalized_external.columns

# assert normalized_external.columns.isin(comp_data['rxn_labels']).all()
# normalized_external.head()

### Convert intracellular concentrations into relative metabolite concentrations

In [15]:
df_internal_mean = df_internal.join(df_internal['Assay Name'].str.extract(
    '^(?P<strain>\S+)-(?P<media>\S+)-(?P<replicate>R\d+)')).pivot_table(
    values='Value', index=['strain', 'media'], columns='metabolite').drop('WT')

df_internal_mean += 1E-6  # Ensure nothing is zero
df_internal_mean.columns = df_internal_mean.columns.str.replace('_[cp]', '')

normalized_internal = df_internal_mean.divide(df_internal_mean.loc[('GB032', 'gfg')])

model_mets = {m.id[:-2] for m in model.metabolites.query('c', 'compartment')}
normalized_internal = normalized_internal.loc[:, normalized_internal.columns.isin(model_mets)]

In [16]:
xn = np.log(normalized_internal)
x_inds = np.array([model.metabolites.index(m + '_c') for m in xn.columns])

# pd.Series(xn.columns).to_csv('temp_data/measured_mets.csv', index=False)
print(xn.shape)

(24, 21)


## Use media measurements to establish y values

In [17]:
media_data = pd.read_csv('media.csv')  # Use quantification of the initial media

media = media_data.set_index('Media')[['Glucose (mM)', 'Fructose (mM)', 'Gluconic acid (mM)']]
media = media.replace(0, .1)

y = media.reindex(index=[index[1] for index in xn.index])
yn = np.log(y)
yn.head()

,Glucose (mM),Fructose (mM),Gluconic acid (mM)
Media,,,
f,-2.302585,3.388712,-2.302585
fg,2.999362,2.808448,-2.302585
gfg,2.502261,2.285358,2.111232
gg,2.983843,-2.302585,2.586933
glc,-2.302585,-2.302585,3.305744


### Convert protein concentrations into relative enzyme expression

In [18]:
def iter_locus_matches():
    
    for locus in tqdm(df_protein.Locus.unique()):
        
        try:
            gene = model.genes.get_by_id(locus)
        except KeyError:
            continue
        
        for reaction in gene.reactions:
            yield pd.Series({'Locus': locus, 'Reaction': reaction.id})

bigg_ids = pd.DataFrame(iter_locus_matches()).astype(str)
df_protein_bigg = df_protein.join(df_protein['Assay Name'].str.extract(
    '^(?P<strain>\S+)-(?P<media>\S+)-(?P<replicate>R\d+)'))

# The existing data is log-transformed, undo the log before calculating mean statistics
df_protein_bigg['Value'] = np.exp(df_protein_bigg['Value'])

locus_means = df_protein_bigg.groupby(['strain', 'media', 'Locus']).Value.mean()
normalized_locus = locus_means.divide(locus_means.loc['GB032', 'gfg'])

norm_with_compressed_rxns = normalized_locus.reset_index().merge(
    bigg_ids, on='Locus', how='left').dropna(subset=['Reaction'])

en = np.log(norm_with_compressed_rxns.groupby
            (['strain', 'media', 'Reaction']).mean()).reset_index().pivot_table(
    values='Value', index=['strain', 'media'], columns='Reaction'
    ).drop('WT').dropna(axis=1).clip(lower=-3, upper=3)

e_inds = np.array([rxn_indexer.loc[r] for r in en.columns]).flatten()

en.head()

100%|██████████| 3098/3098 [00:00<00:00, 27389.54it/s]


Reaction       2DHGLCK    ACONTa    ACONTb     AKGDH    CATDOX        CS  \
strain media                                                               
CJ522  f     -3.000000  0.272017  0.272017 -0.241436  1.311682 -0.604015   
       fg    -3.000000  0.213284  0.213284 -0.073110  0.687598 -0.266162   
       gfg   -0.568328  0.134530  0.134530  0.044835  0.237304  0.021126   
       gg     1.375045  0.048980  0.048980 -0.209050  0.297953 -0.238629   
       glc    2.199031 -0.050325 -0.050325 -0.008460  0.384754  0.220045   

Reaction          DDPA      DHQD      DHQS    DHSKDH  ...        PC       PDH  \
strain media                                          ...                       
CJ522  f      3.000000 -1.039744  0.227510  0.574359  ... -0.441525  0.264335   
       fg     0.075271  0.187655  0.031240  0.455298  ... -0.059478 -0.140848   
       gfg    0.185219 -0.480403  0.187833  0.405855  ...  0.023156  0.065905   
       gg     0.031278  0.940062 -0.010494  0.382123  ...  0.508754  0.165245   
       glc    3.000000  0.711716  0.355852  0.537603  ...  0.805512  2.148161   

Reaction       PGLCNDH       RPE       RPI     SUCDi      TALA      TKT1  \
strain media                                                               
CJ522  f     -3.000000 -0.323297 -0.072284 -0.097492  0.142874 -0.188608   
       fg    -2.989562 -0.189002  0.150008 -0.047481 -0.140595  0.061767   
       gfg   -0.471832 -0.131052  0.114781  0.086695 -0.031795 -0.073953   
       gg     1.245499 -0.082311  0.190140 -0.178901 -0.078044 -0.105722   
       glc    1.461438 -0.058035  0.040824  0.035145 -0.129291  0.018410   

Reaction          TKT2       TPI  
strain media                      
CJ522  f     -0.188608 -0.019851  
       fg     0.061767  0.065636  
       gfg   -0.073953  0.119347  
       gg    -0.105722 -0.060508  
       glc    0.018410  0.075858  

[5 rows x 46 columns]

In [19]:
# some enzymes are unmeasured but can vary, others we want to pin at zero
e_laplace_inds = []
e_zero_inds = []

for i, rxnid in enumerate(r_labels):
    rxn = model.reactions.get_by_id(rxnid)
    if rxnid not in en.columns:
#        e_laplace_inds += [i]
        
        if 'e' not in rxn.compartments:
            e_laplace_inds += [i]
        else:
            e_zero_inds += [i]

e_laplace_inds = np.array(e_laplace_inds)
e_zero_inds = np.array(e_zero_inds)
e_indexer = np.hstack([e_inds, e_laplace_inds, e_zero_inds]).argsort()

In [20]:
xn.head()

metabolite     2dhglcn    34dhbz       3pg      6pgc  catechol    ccmuac  \
strain media                                                               
CJ522  f     -1.507983 -0.516841  0.188159  1.573144 -0.197256  0.146698   
       fg    -2.292114 -0.609229 -0.517821 -0.418059 -0.258957 -0.326042   
       gfg   -0.325099 -0.477608 -0.198279  2.648919  0.364816  0.077610   
       gg     0.609105  0.565875 -1.176226 -0.711994  1.022891  0.401051   
       glc    1.127845 -0.193345 -1.071695  0.452522  0.418890  0.374417   

metabolite         cit      dhap       e4p       fum  ...    glc__D  \
strain media                                          ...             
CJ522  f     -0.060643  0.223848  1.110105 -0.701411  ... -0.053046   
       fg    -0.641151 -0.524034 -1.258021 -1.148938  ...  1.242978   
       gfg   -0.233806  0.204713  0.620121 -0.097070  ...  0.269630   
       gg     0.572711 -0.483895 -1.545005  0.349620  ...  1.399985   
       glc    0.373159 -0.430374 -2.617502  0.631929  ... -3.085443   

metabolite         glcn    gln__L    glu__L      hco3       pep        pi  \
strain media                                                                
CJ522  f       0.000000  0.472526  1.326683  0.066313 -0.475707  0.121436   
       fg      0.000000 -0.922868 -0.751181 -0.001099  0.142129  0.001032   
       gfg    25.325444  0.184214  1.701217 -0.095145 -0.486271 -0.020035   
       gg     26.973226 -0.613969  1.580109 -0.000494 -0.708132 -0.001610   
       glc    29.642370  0.002317  2.320946  0.068902 -0.064614 -0.020042   

metabolite         pyr   ru5p__D       s7p  
strain media                                
CJ522  f     -0.012915  0.217387  0.261399  
       fg    -0.756668 -0.783303 -0.845496  
       gfg    0.105480  0.613403  0.374904  
       gg     0.711496 -0.395079 -0.309882  
       glc    0.957726 -0.016646 -0.111536  

[5 rows x 21 columns]

## Construct the probablity model

In [21]:
import pymc3 as pm

import theano
import theano.tensor as T
from theano import sparse

import emll
from emll.util import initialize_elasticity

In [22]:
Ey = np.zeros((len(model.reactions), 3))
Ey[model.reactions.index('EX_glc_e'), 0] = 1
Ey[model.reactions.index('EX_fru_e'), 1] = 1
Ey[model.reactions.index('EX_glcn_e'), 2] = 1

ex_labels = np.array([['$\epsilon_{' + '{0},{1}'.format(rlabel, mlabel) + '}$'
                       for mlabel in m_labels] for rlabel in r_labels]).flatten()

r_compartments = [
    list(r.compartments)[0] if len(r.compartments) == 1 else 't'
    for r in model.reactions
]

m_compartments = [
    m.compartment for m in model.metabolites
]

ll = emll.LinLogLeastNorm(N, -N.T, Ey, v_star)

n_exp = xn.shape[0]
n_exp

24

In [23]:
with pm.Model() as pymc_model:
    
    # Initialize elasticities
    Ex_t = pm.Deterministic(
        'Ex', initialize_elasticity(N, 'ex', b=0.05, sd=1, alpha=None,
                                    m_compartments=m_compartments,
                                    r_compartments=r_compartments))
                                                        
    Ey_t = pm.Deterministic('Ey', initialize_elasticity(-Ey.T, 'ey', b=0.05, sd=1, alpha=None))
    
    e_measured = pm.Normal('log_e_measured', mu=en.values, sd=0.2,
                           shape=(n_exp, len(e_inds)))
    e_unmeasured = pm.Laplace('log_e_unmeasured', mu=0, b=0.1,
                              shape=(n_exp, len(e_laplace_inds)))
    log_en_t = T.concatenate(
        [e_measured, e_unmeasured,
         T.zeros((n_exp, len(e_zero_inds)))], axis=1)[:, e_indexer]

    pm.Deterministic('log_en_t', log_en_t)
    
    yn_t = T.as_tensor_variable(yn.values)
    
    chi_ss, vn_ss = ll.steady_state_theano(Ex_t, Ey_t, T.exp(log_en_t), yn_t)
    pm.Deterministic('chi_ss', chi_ss)
    pm.Deterministic('vn_ss', vn_ss)
    
#     vn_subset = T.clip(vn_ss[:, v_inds], 0, 2)
    
    chi_clip = T.clip(chi_ss[:, x_inds], -3, 3)  

    chi_obs = pm.Normal('chi_obs', mu=chi_clip, sd=0.2,
                        observed=xn.clip(lower=-3, upper=3).values)
    
    vn_obs = pm.Normal('vn_obs', mu=vn_ss[:, v_inds], sd=0.025,
                           observed=vn.values)
    
print(pymc_model.logpt.tag.test_value)

-976607.0825634664


In [24]:
with gzip.open('model.pz', 'wb') as f:
    pickle.dump(pymc_model, f)

In [29]:
with gzip.open('model_data.pz', 'wb') as f:
    pickle.dump({
        'model': model,
        'vn': vn,
        'en': en,
        'yn': yn,
        'xn': xn,
        'x_inds': x_inds,
        'e_inds': e_inds,
        'v_inds': v_inds,
        'm_labels': m_labels,
        'r_labels': r_labels,
        'll': ll}
        , f)